In [ ]:
# Try my own code to find ATP generating cycles in the model

In [1]:
import cobra

/Users/helenscott/opt/miniconda3/envs/med4-hot1a3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
model = cobra.io.read_sbml_model('model.xml')

'' is not a valid SBML 'SId'.


In [13]:
# Add disspation reaction the same way that MEMOTE does
# Make a reaction with water as the reactant, and H+ and phosphate as the products
model.add_reactions([cobra.Reaction('dissipation')])
model.reactions.get_by_id('dissipation').add_metabolites({model.metabolites.get_by_id('cpd00001_c0'):-1,
                                                            model.metabolites.get_by_id('cpd00067_c0'):1,
                                                            model.metabolites.get_by_id('cpd00009_c0'):1})

In [16]:
model.reactions.dissipation.lower_bound = -1000
model.reactions.dissipation

Reaction identifier,dissipation
Name,
Memory address,0x169702860
Stoichiometry,cpd00001_c0 <=> cpd00009_c0 + cpd00067_c0 H2O [c0] <=> Phosphate [c0] + H+ [c0]
GPR,
Lower bound,-1000
Upper bound,1000.0


In [10]:
# Block all exchange reactions excpet for water
media = {'EX_cpd00001_e0': 1000}
model.medium = media

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


In [4]:
# A function that checks if a specific reaction is a maintenace reaction
# by checking if the reaction has only ATP and water as reactants and
# only ADP, hydrogen, and phosphate as products.
def is_maintenance_reaction(model, reaction):
    # A list of the IDs of the reactants and products that are allowed
    # in a maintenance reaction. Have to specify the compartment of the
    # metabolites, this may be a problem if different models use
    # different compartment IDs.
    # ATP and water are allowed reactants.
    allowed_reactants = ['cpd00002_c0', 'cpd00001_c0']
    # ADP, phosphate, and protons are allowed products.
    allowed_products = ['cpd00008_c0', 'cpd00009_c0', 'cpd00067_c0']
    # Check if the reaction has only the allowed reactants and products
    # in any order.
    if any (met.id not in allowed_reactants for met in reaction.reactants):
        return False
    if any (met.id not in allowed_products for met in reaction.products):
        return False
    else:
        return True

# A function that searches all the reactions in a model for a maintenance
# reaction, and returns the reaction if it finds one.
def find_maintenance_reaction(model):
    maintenace_rxns = []
    for reaction in model.reactions:
        if is_maintenance_reaction(model, reaction):
            maintenace_rxns.append(reaction)
    # If there is only one maintenance reaction, return it.
    if len(maintenace_rxns) == 1:
        return maintenace_rxns[0]
    # If there are no maintenance reactions, return None.
    elif len(maintenace_rxns) == 0:
        return None
    # If there are multiple maintenance reactions, return a list of them
    # and print a warning.
    else:
        print('Warning: multiple maintenance reactions found.')
        return maintenace_rxns

In [6]:
# Do I have a maintenance reaction
atpm = find_maintenance_reaction(model)[0]
atpm

Reaction identifier,rxn00062_c0
Name,ATP hydrolysis
Memory address,0x176165a50
Stoichiometry,cpd00001_c0 + cpd00002_c0 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 H2O [c0] + ATP [c0] --> ADP [c0] + Phosphate [c0] + H+ [c0]
GPR,
Lower bound,0.0
Upper bound,1000.0


In [7]:
# Set the objective to be the ATP maintenance reaction
model.objective = atpm

In [18]:
# Run and get just the non-zero fluxes
sol = model.optimize()
fluxes = sol.fluxes[sol.fluxes != 0]
print(fluxes)

rxn05312_c0      -1000.0
rxn08173_c0        250.0
rxn05319_c0       1000.0
rxn00062_c0        250.0
EX_cpd00009_e0    1000.0
EX_cpd00001_e0   -1000.0
DM_cpd00067_c0    1000.0
dissipation       1000.0
Name: fluxes, dtype: float64


In [19]:
model.reactions.rxn08173_c0

Reaction identifier,rxn08173_c0
Name,F(1)-ATPase [c0]
Memory address,0x176094a60
Stoichiometry,cpd00008_c0 + cpd00009_c0 + 4.0 cpd00067_e0 <=> cpd00001_c0 + cpd00002_c0 + 3.0 cpd00067_c0 ADP [c0] + Phosphate [c0] + 4.0 H+ [e0] <=> H2O [c0] + ATP [c0] + 3.0 H+ [c0]
GPR,MIT1002_04119 and MIT1002_04115 and MIT1002_04117 and MIT1002_04113 and MIT1002_04120 and...
Lower bound,-1000.0
Upper bound,1000.0


In [21]:
model.reactions.rxn05595_c0

Reaction identifier,rxn05595_c0
Name,TRANS-RXN-3.ce [c0]
Memory address,0x175806860
Stoichiometry,cpd00067_e0 + cpd00205_e0 <=> cpd00067_c0 + cpd00205_c0 H+ [e0] + K+ [e0] <=> H+ [c0] + K+ [c0]
GPR,MIT1002_00016 or MIT1002_00017
Lower bound,-1000.0
Upper bound,1000.0


In [22]:
model.reactions.rxn05206_c0

Reaction identifier,rxn05206_c0
Name,TRANS-RXN-187.ce [c0]
Memory address,0x175f49060
Stoichiometry,cpd00205_e0 <-- cpd00205_c0 K+ [e0] <-- K+ [c0]
GPR,MIT1002_03754 and MIT1002_01599 and MIT1002_01761 and MIT1002_03343 and MIT1002_02492
Lower bound,-1000.0
Upper bound,0.0
